# 1. Install unsloth and related packages

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install unsloth
# Get latest Unsloth
!pip install --upgrade --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
#     "unsloth/llama-3-8b-Instruct-bnb-4bit",
#     "unsloth/llama-3-70b-bnb-4bit",
#     "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.10: Fast Llama patching. Transformers: 4.52.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.10 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# Data Prep

In [ ]:
from datasets import load_dataset
dataset = load_dataset(
    "csv",
    data_files = "https://huggingface.co/datasets/unsloth/datasets/raw/main/titanic.csv",
    split = "train",
)
print(dataset.column_names)
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
{'PassengerId': 1, 'Survived': 0, 'Pclass': 3, 'Name': 'Braund, Mr. Owen Harris', 'Sex': 'male', 'Age': 22.0, 'SibSp': 1, 'Parch': 0, 'Ticket': 'A/5 21171', 'Fare': 7.25, 'Cabin': None, 'Embarked': 'S'}


In [ ]:
print(dataset.column_names)

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [ ]:
# Split into train and test (80-20 split)
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

print(f"Train size: {len(train_dataset)}")
print(f"Test size: {len(test_dataset)}")

Train size: 712
Test size: 179


In [ ]:
# from unsloth import to_sharegpt
# dataset_simple = to_sharegpt(
#     dataset,
#     merged_prompt = "[[Their age is {Age}.\n]][[They paid ${Fare} for the trip.\n]]",
#     output_column_name = "Survived",
# )

In [ ]:
from unsloth import to_sharegpt
train_dataset = to_sharegpt(
    train_dataset,
    merged_prompt = \
        "[[The passenger embarked from {Embarked}.]]"\
        "[[\nThey are {Sex}.]]"\
        "[[\nThey have {Parch} parents and childen.]]"\
        "[[\nThey have {SibSp} siblings and spouses.]]"\
        "[[\nTheir passenger class is {Pclass}.]]"\
        "[[\nTheir age is {Age}.]]"\
        "[[\nThey paid ${Fare} for the trip.]]",
    conversation_extension = 5, # Randomnly combines conversations into 1! Good for long convos
    output_column_name = "Survived",
)

Merging columns:   0%|          | 0/712 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/712 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/712 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/712 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/712 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/712 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/712 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/712 [00:00<?, ? examples/s]

In [ ]:
from pprint import pprint
pprint(train_dataset[0])

{'conversations': [{'from': 'human',
                    'value': 'The passenger embarked from S.\n'
                             'They are male.\n'
                             'Their passenger class is 3.\n'
                             'Their age is 22.0.\n'
                             'They paid $7.125 for the trip.'},
                   {'from': 'gpt', 'value': '0'},
                   {'from': 'human',
                    'value': 'The passenger embarked from C.\n'
                             'They are female.\n'
                             'They have 1 siblings and spouses.\n'
                             'Their passenger class is 2.\n'
                             'Their age is 27.0.\n'
                             'They paid $13.8583 for the trip.'},
                   {'from': 'gpt', 'value': '1'},
                   {'from': 'human',
                    'value': 'The passenger embarked from S.\n'
                             'They are male.\n'
                            

In [ ]:
from unsloth import standardize_sharegpt
train_dataset = standardize_sharegpt(train_dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/712 [00:00<?, ? examples/s]

In [ ]:
train_dataset[0]

{'conversations': [{'content': 'The passenger embarked from S.\nThey are male.\nTheir passenger class is 3.\nTheir age is 22.0.\nThey paid $7.125 for the trip.',
   'role': 'user'},
  {'content': '0', 'role': 'assistant'},
  {'content': 'The passenger embarked from C.\nThey are female.\nThey have 1 siblings and spouses.\nTheir passenger class is 2.\nTheir age is 27.0.\nThey paid $13.8583 for the trip.',
   'role': 'user'},
  {'content': '1', 'role': 'assistant'},
  {'content': 'The passenger embarked from S.\nThey are male.\nTheir passenger class is 1.\nTheir age is 61.0.\nThey paid $33.5 for the trip.',
   'role': 'user'},
  {'content': '0', 'role': 'assistant'},
  {'content': 'The passenger embarked from S.\nThey are male.\nTheir passenger class is 2.\nTheir age is 19.0.\nThey paid $10.5 for the trip.',
   'role': 'user'},
  {'content': '1', 'role': 'assistant'},
  {'content': 'The passenger embarked from S.\nThey are male.\nTheir passenger class is 3.\nThey paid $9.5 for the trip.',

In [ ]:
chat_template = """Below describes some details about some passengers who went on the Titanic.
Predict whether they survived or perished based on their characteristics.
Output 1 if they survived, and 0 if they died.
>>> Passenger Details:
{INPUT}
>>> Did they survive?
{OUTPUT}"""

from unsloth import apply_chat_template
train_dataset = apply_chat_template(
    train_dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/712 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to=[]
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/712 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 712 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.614700
2,1.562900
3,1.551200
4,1.438600
5,1.313700
6,1.169600
7,1.011400
8,0.956200
9,0.870800
10,0.780700


In [ ]:
FastLanguageModel.for_inference(model) # Comment this out
messages = [                    # Change below!
    {"role": "user", "content": 'The passenger embarked from S.\n'\
                                'They are male.\n'\
                                'They have 1 siblings and spouses.\n'\
                                'Their passenger class is 3.\n'\
                                'Their age is 22.0.\n'\
                                'They paid $7.25 for the trip.'},
]
device = "cuda" if torch.cuda.is_available() else "cpu"
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to(device)



In [ ]:
# Set up streaming generation
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

In [ ]:
outputs = model.generate(
    input_ids,
    streamer=text_streamer,
    max_new_tokens=128,
    pad_token_id=tokenizer.eos_token_id,
    use_cache=False,  # Disable caching
    attention_mask=torch.ones_like(input_ids)  # Explicit attention mask
)

The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


0<|end_of_text|>


# Test Data

In [ ]:
# Convert test dataset to the same format as training data
test_dataset_formatted = to_sharegpt(
    test_dataset,
    merged_prompt = \
        "[[The passenger embarked from {Embarked}.]]"\
        "[[\nThey are {Sex}.]]"\
        "[[\nThey have {Parch} parents and childen.]]"\
        "[[\nThey have {SibSp} siblings and spouses.]]"\
        "[[\nTheir passenger class is {Pclass}.]]"\
        "[[\nTheir age is {Age}.]]"\
        "[[\nThey paid ${Fare} for the trip.]]",
    conversation_extension = 1,  # Keep it 1 for individual predictions
    output_column_name = "Survived",
)


Merging columns:   0%|          | 0/179 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/179 [00:00<?, ? examples/s]

In [ ]:
# Process each test sample
predictions = []
actual_labels = []

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Helper function to extract prediction from model output
def extract_prediction(response):
    """Extract 0 or 1 from model response"""
    response = response.strip().lower()
    if '1' in response:
        return 1
    elif '0' in response:
        return 0
    else:
        # Handle edge cases where model doesn't give clear 0/1
        if 'survived' in response or 'yes' in response:
            return 1
        else:
            return 0

In [ ]:
for i, sample in enumerate(test_dataset_formatted):
    # Extract the input content (without the expected output)
    user_content = sample['conversations'][0]['value']  # The formatted passenger details

    messages = [
        {"role": "user", "content": user_content}
    ]

    # Tokenize
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(device)

    # Generate prediction
    with torch.no_grad():
      outputs = model.generate(
            input_ids,
            streamer=text_streamer,
            max_new_tokens=10,  # Short response needed (just 0 or 1)
            pad_token_id=tokenizer.eos_token_id,
            use_cache=False,  # Use greedy decoding for consistent results
            temperature=0.1,
            attention_mask=torch.ones_like(input_ids)  # Explicit attention mask
        )
    # Decode the response
    response = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)

    # Extract prediction (0 or 1)
    predicted_label = extract_prediction(response)
    actual_label = sample['conversations'][1]['value']  # The actual survival status

    predictions.append(predicted_label)
    actual_labels.append(int(actual_label))

    # print(f"Sample {i+1}: Predicted={predicted_label}, Actual={actual_label}")

The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


0<|end_of_text|>


The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


0<|end_of_text|>


The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

0<|end_of_text|>


The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


0<|end_of_text|>


The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


0<|end_of_text|>


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Simple text-based confusion matrix with percentages
def print_detailed_confusion_matrix(actual, predicted):
    cm = confusion_matrix(actual, predicted)
    tn, fp, fn, tp = cm.ravel()
    total = len(actual)

    print("\nDetailed Confusion Matrix:")
    print("="*40)
    print(f"{'':12} {'Predicted':>20}")
    print(f"{'':12} {'Died':>8} {'Survived':>8} {'Total':>8}")
    print("-"*40)
    print(f"{'Died':12} {tn:5d} ({tn/total*100:4.1f}%) {fp:5d} ({fp/total*100:4.1f}%) {tn+fp:5d}")
    print(f"{'Survived':12} {fn:5d} ({fn/total*100:4.1f}%) {tp:5d} ({tp/total*100:4.1f}%) {fn+tp:5d}")
    print("-"*40)
    print(f"{'Total':12} {tn+fn:5d} {fp+tp:5d} {total:5d}")

    print(f"\nModel Performance:")
    print(f"Correctly predicted deaths: {tn}/{tn+fp} = {tn/(tn+fp)*100:.1f}%")
    print(f"Correctly predicted survivals: {tp}/{fn+tp} = {tp/(fn+tp)*100:.1f}%")
    print(f"Overall accuracy: {(tn+tp)/total*100:.1f}%")

# Call the function
print_detailed_confusion_matrix(actual_labels, predictions)


Detailed Confusion Matrix:
                        Predicted
                 Died Survived    Total
----------------------------------------
Died            98 (54.7%)     1 ( 0.6%)    99
Survived        61 (34.1%)    19 (10.6%)    80
----------------------------------------
Total          159    20   179

Model Performance:
Correctly predicted deaths: 98/99 = 99.0%
Correctly predicted survivals: 19/80 = 23.8%
Overall accuracy: 65.4%


In [ ]:
# Decode the response
response = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)

# Extract prediction (0 or 1)
predicted_label = extract_prediction(response)
actual_label = sample['conversations'][1]['value']  # The actual survival status

predictions.append(predicted_label)
actual_labels.append(int(actual_label))

print(f"Sample {i+1}: Predicted={predicted_label}, Actual={actual_label}")

Sample 179: Predicted=0, Actual=1
